# Libraries

In [32]:
SEED=2023

import numpy as np
import pandas as pd
import os
import sys
import math
import random
import gc
import pathlib
from typing import List, NoReturn, Union, Tuple, Optional, Text, Generic, Callable, Dict
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, mean_squared_error, mean_absolute_error, f1_score
from scipy.stats import spearmanr

import tensorflow as tf
!pip install numerapi
import numerapi
# import tensorflow_addons as tfa
# !pip install -q -U keras-tuner
!pip install keras-tuner
import keras_tuner as kt # keras tuner!

import zipfile

# visualize
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
sns.set_context("talk")
style.use('fivethirtyeight')

data_directory = "../kazutsugi/datasets/"
EXP_DIR ="../example"
OUTPUT_DIR= "../output"


You should consider upgrading via the '/Users/andrewjennings/PycharmProjects/numerai_tree_regression/venv/tree_regression/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/andrewjennings/PycharmProjects/numerai_tree_regression/venv/tree_regression/bin/python -m pip install --upgrade pip' command.


In [33]:
!pip install numerapi

^C


# Config
Here you can choose what you do: 

- Debug mode (using small proportion of data)?
- Tuning or not?
- Which target to predict?
- Seed number?

In [34]:
class CFG:
    DEBUG = False # debug option
    TUNING = True # whether to use the KerasTuner or not

    INPUT_FILE_PATH = '../input/numerai-train-validation-with-kazutsugi-nomi/numerai_training_validation_target_nomi.csv'
    OUTPUT_DIR = ''
    TARGET = 'target_nomi' # target_kazutsugi
    SEED = 2021

In [35]:
# Logging is always nice for your experiment:)
def init_logger(log_file=CFG.OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

logger = init_logger()
logger.info('Start Logging...')

Start Logging...
Start Logging...
Start Logging...
2021-10-16 15:20:43,565 INFO __main__: Start Logging...


In [36]:
logger.info('DEBUG : {}'.format(CFG.DEBUG))
logger.info('TUNING : {}'.format(CFG.TUNING))
logger.info('TARGET : {}'.format(CFG.TARGET))

DEBUG : False
DEBUG : False
DEBUG : False
2021-10-16 15:20:43,572 INFO __main__: DEBUG : False
TUNING : True
TUNING : True
TUNING : True
2021-10-16 15:20:43,575 INFO __main__: TUNING : True
TARGET : target_nomi
TARGET : target_nomi
TARGET : target_nomi
2021-10-16 15:20:43,577 INFO __main__: TARGET : target_nomi


# Load data
In this dataset, we have two targets: 'kazutsugi' (old one) and 'nomi' (new one).

In [37]:
def download_data():

    napi = numerapi.NumerAPI(verbosity="info")
    data_archive = napi.download_current_dataset(dest_path='../tmp', unzip=False)

    with zipfile.ZipFile(data_archive, "r") as zip_ref:
        zip_ref.extractall("../kazutsugi/datasets")





In [38]:
def get_data():

    download_data()

    print("# Loading data...")
    # The training data is used to train your model how to predict the targets.
    training_data = pd.read_csv(data_directory + "numerai_training_data.csv").set_index("id")
    # The tournament data is the data that Numerai uses to evaluate your model.
    tournament_data = pd.read_csv(data_directory + "numerai_tournament_data.csv").set_index("id")

    feature_names = [ f for f in training_data.columns if f.startswith("feature")]

    print(f"Loaded {len(feature_names)} features")


    print("Training model")



    return training_data,feature_names,tournament_data

In [39]:
%%time

def get_int(x):
    try:
        return int(x[3:])
    except:
        return 1000

training_data,feature_names,tournament_data = get_data()
validation_data = tournament_data[tournament_data.data_type == "validation"]

2021-10-16 15:20:44,937 INFO numerapi.utils: target file already exists
2021-10-16 15:20:44,938 INFO numerapi.utils: resuming download


KeyboardInterrupt: 

In [40]:
print(training_data.shape)
training_data.head()

NameError: name 'training_data' is not defined

In [ ]:
print(validation_data.shape)
validation_data.head()

In [ ]:
# debug 
if CFG.DEBUG:
    training_data = training_data.sample(10000, random_state=SEED)
    logger.info('reduced train for debug')

# Features, Target
We use all the features for now. As for a target, you specify which one to use before in the config section.

In [ ]:
# features
features = training_data.columns[training_data.columns.str.startswith('feature')].values.tolist()
logger.info('{:,} features.'.format(len(features)))
logger.info(features)

In [ ]:
# target
sns.jointplot(data=training_data, x="target_kazutsugi", y="target_nomi")

# Modeling
Here we build a simple MLP using tensorflow.

In [ ]:
# set seed to reproduce the result
def seed_everything(seed : int) -> NoReturn :    
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

seed_everything(CFG.SEED) 

In [ ]:
# my default NN hyperparameters
params = {
    'input_dim': len(features),
    'input_dropout': 0.0,
    'hidden_layers': 3,
    'hidden_units': 256,
    'hidden_activation': 'relu',
    'lr': 1e-03,
    'dropout': 0.2,
    'batch_size': 128,
    'epochs': 192
}
logger.info('default NN params:')
logger.info(params)

def create_model(params=params):
    """
    baseline model
    """

    # NN model architecture
    n_neuron = params['hidden_units']

    inputs = tf.keras.layers.Input(shape=(params['input_dim'], ))
    x = tf.keras.layers.BatchNormalization()(inputs)
    x = tf.keras.layers.Dense(n_neuron, activation=params['hidden_activation'])(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(params['dropout'])(x)

    # stack more layers
    for i in np.arange(params['hidden_layers'] - 1):
        x = tf.keras.layers.Dense(n_neuron // (2 * (i+1)), activation=params['hidden_activation'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(params['dropout'])(x)

    out = tf.keras.layers.Dense(1, activation='linear', name = 'out')(x)
        
    # compile
    model = tf.keras.models.Model(inputs=inputs, outputs=out)
    loss = tf.keras.losses.MeanSquaredError()
    opt = tf.keras.optimizers.Adam(lr=params['lr'])
    model.compile(loss=loss, optimizer=opt, metrics=['mse'])
    
    return model

def tuning_model(hp, params=params):
    """
    model tuning with KerasTuner
    """
    
    inputs = tf.keras.layers.Input(shape=(params['input_dim'], ))
    x = tf.keras.layers.BatchNormalization()(inputs)
    x = tf.keras.layers.Dense(hp.Int('num_units_1', 128, 512, step=128), activation=params['hidden_activation'])(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(hp.Float('dropout_1', 0.0, 0.5, step=0.1, default=0.5))(x)

    for i in range(hp.Int('num_layers', 1, 3)):
        x = tf.keras.layers.Dense(hp.Int(f'num_units_{i+2}', 128, 512, step=128))(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(hp.Float(f'dropout_{i+2}', 0.0, 0.5, step=0.1, default=0.5))(x)
        
    # output
    out = tf.keras.layers.Dense(1, activation='linear', name = 'out')(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=out)
   
    # compile
    loss = tf.keras.losses.MeanSquaredError()
    opt = tf.keras.optimizers.Adam(lr=hp.Float('learning_rate', 1e-4, 1e-2, sampling='log'))
    model.compile(loss=loss, optimizer=opt, metrics=['mse'])
    
    return model


# Tuning (or not)
Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - RandomSearch, Hyperband, BayesianOptimization, and Sklearn. 

Here we use the **BayesianOptimization** tuner.

In [ ]:
# create a dataset for NN based on a task
train_set = {'X': training_data[features].values, 'y': training_data[CFG.TARGET].values}
valid_set = {'X': validation_data[features].values, 'y': validation_data[CFG.TARGET].values}

In [ ]:
if CFG.TUNING:
    # define a custom tuner to tune the batch size
    class MyTuner(kt.tuners.BayesianOptimization):
      def run_trial(self, trial, *args, **kwargs):
        # You can add additional HyperParameters for preprocessing and custom training loops
        # via overriding `run_trial`
        kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 128, 8192, step=128)
#         kwargs['epochs'] = trial.hyperparameters.Int('epochs', 10, 30)
        super(MyTuner, self).run_trial(trial, *args, **kwargs)

    # instantiate KerasTuner
    model_ft = lambda hp: tuning_model(hp, params)
    tuner = MyTuner(
        hypermodel=model_ft,
        objective=kt.Objective('val_loss', direction='min'),
        num_initial_points=4,
        max_trials=20,
        overwrite=True)
    
    # perform tuning
    tuner.search(train_set['X'], train_set['y'], 
                 epochs = 8, validation_data = (valid_set['X'], valid_set['y']))

    # Get the optimal hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
    
    # Build the model with the optimal hyperparameters and train it on the data
    model = tuner.hypermodel.build(best_hps)
    
    # disp best params
    logger.info('Best hyperparameters:')
    logger.info(best_hps.values)
else:
    # baseline (no tuning)
    model = create_model(params)

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

# Fit with the best model
Let's use the model with the best hyperparameters (if tuned). 

Here I use **Early Stopping** such that the model does not overfit. 

As a learning rate scheduler, I use **ReduceLROnPlateau**.

I do not submit the model prediction in this notebook, but to make the submission process a bit easier, I save the entire model!

In [ ]:
%%time

# callbacks
es = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True, monitor='val_loss')
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, verbose=1, mode='min')
# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='mybestweight.hdf5', save_weights_only=True, 
#                                                                verbose=0, monitor='val_loss', save_best_only=True)
nn_callbacks = [es, lr_scheduler, ]

# fit
history = model.fit(train_set['X'], train_set['y'], callbacks=nn_callbacks, 
                    verbose=2, epochs=params['epochs'], batch_size=best_hps.values['batch_size'], 
                    validation_data=(valid_set['X'], valid_set['y'])) 

# you can load the model for inference
# model = tf.keras.models.load_model(CFG.OUTPUT_DIR + 'saved_model/my_model')

In [ ]:
# save the entire model
model.save(CFG.OUTPUT_DIR + 'my_model.h5')
logger.info('Entire model saved!')

In [ ]:
# summarize history for loss
with plt.xkcd(): # just for fun
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.show()

# Validation Prediction

In [ ]:
# prediction for valid
pred = model.predict(valid_set['X']).ravel()

In [ ]:
plt.hist(pred, alpha=0.4, label='prediction')
plt.hist(valid_set['y'], alpha=0.4, label='actual target')
plt.legend(frameon=False)

# Validation Score
This is what we care about! Here we compute Numerai-related scores except for MMC (which we cannot compute as we don't have a meta-model prediction).

Note that we split the validation set into the two parts and compute scores on the corresponding eras. This is because the first half validation eras are easy to predict, whereas the last half are hard. It is better to see that our model performs well on the both periods.

In [ ]:
# naming conventions
PREDICTION_NAME = 'prediction'
TARGET_NAME = CFG.TARGET
EXAMPLE_PRED = 'example_prediction'

# ---------------------------
# Functions
# ---------------------------
def valid4score(valid : pd.DataFrame, pred : np.ndarray, load_example: bool=True, save : bool=False) -> pd.DataFrame:
    """
    Generate new valid pandas dataframe for computing scores
    
    :INPUT:
    - valid : pd.DataFrame extracted from tournament data (data_type='validation')
    
    """
    valid_df = valid.copy()
    valid_df['prediction'] = pd.Series(pred).rank(pct=True, method="first")
    valid_df.rename(columns={TARGET_NAME: 'target'}, inplace=True)
    
    if load_example:
        valid_df[EXAMPLE_PRED] = pd.read_csv(EXP_DIR + 'valid_df.csv')['prediction'].values
    
    if save==True:
        valid_df.to_csv(OUTPUT_DIR + 'valid_df.csv', index=False)
        logger.info('Validation dataframe saved!')
    
    return valid_df

def compute_corr(valid_df : pd.DataFrame):
    """
    Compute rank correlation
    
    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    
    """
    
    return np.corrcoef(valid_df["target"], valid_df['prediction'])[0, 1]

def compute_max_drawdown(validation_correlations : pd.Series):
    """
    Compute max drawdown
    
    :INPUT:
    - validation_correaltions : pd.Series
    """
    
    rolling_max = (validation_correlations + 1).cumprod().rolling(window=100, min_periods=1).max()
    daily_value = (validation_correlations + 1).cumprod()
    max_drawdown = -(rolling_max - daily_value).max()
    
    return max_drawdown

def compute_val_corr(valid_df : pd.DataFrame):
    """
    Compute rank correlation for valid periods
    
    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """
    
    # all validation
    correlation = compute_corr(valid_df)
    logger.info("rank corr = {:.4f}".format(correlation))
    return correlation
    
def compute_val_sharpe(valid_df : pd.DataFrame):
    """
    Compute sharpe ratio for valid periods
    
    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """
    # all validation
    d = valid_df.groupby('era')[['target', 'prediction']].corr().iloc[0::2,-1].reset_index()
    me = d['prediction'].mean()
    sd = d['prediction'].std()
    max_drawdown = compute_max_drawdown(d['prediction'])
    logger.info('sharpe ratio = {:.4f}, corr mean = {:.4f}, corr std = {:.4f}, max drawdown = {:.4f}'.format(me / sd, me, sd, max_drawdown))
    
    return me / sd, me, sd, max_drawdown
    
def feature_exposures(valid_df : pd.DataFrame):
    """
    Compute feature exposure
    
    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """
    feature_names = [f for f in valid_df.columns
                     if f.startswith("feature")]
    exposures = []
    for f in feature_names:
        fe = spearmanr(valid_df['prediction'], valid_df[f])[0]
        exposures.append(fe)
    return np.array(exposures)

def max_feature_exposure(fe : np.ndarray):
    return np.max(np.abs(fe))

def feature_exposure(fe : np.ndarray):
    return np.sqrt(np.mean(np.square(fe)))

def compute_val_feature_exposure(valid_df : pd.DataFrame):
    """
    Compute feature exposure for valid periods
    
    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """
    # all validation
    fe = feature_exposures(valid_df)
    fe1, fe2 = feature_exposure(fe), max_feature_exposure(fe)
    logger.info('feature exposure = {:.4f}, max feature exposure = {:.4f}'.format(fe1, fe2))
     
    return fe1, fe2

# to neutralize a column in a df by many other columns
def neutralize(df, columns, by, proportion=1.0):
    scores = df.loc[:, columns]
    exposures = df[by].values

    # constant column to make sure the series is completely neutral to exposures
    exposures = np.hstack(
        (exposures,
         np.asarray(np.mean(scores)) * np.ones(len(exposures)).reshape(-1, 1)))

    scores = scores - proportion * exposures.dot(
        np.linalg.pinv(exposures).dot(scores))
    return scores / scores.std()


# to neutralize any series by any other series
def neutralize_series(series, by, proportion=1.0):
    scores = series.values.reshape(-1, 1)
    exposures = by.values.reshape(-1, 1)

    # this line makes series neutral to a constant column so that it's centered and for sure gets corr 0 with exposures
    exposures = np.hstack(
        (exposures,
         np.array([np.mean(series)] * len(exposures)).reshape(-1, 1)))

    correction = proportion * (exposures.dot(
        np.linalg.lstsq(exposures, scores, rcond=None)[0]))
    corrected_scores = scores - correction
    neutralized = pd.Series(corrected_scores.ravel(), index=series.index)
    return neutralized


def unif(df):
    x = (df.rank(method="first") - 0.5) / len(df)
    return pd.Series(x, index=df.index)

def get_feature_neutral_mean(df):
    feature_cols = [c for c in df.columns if c.startswith("feature")]
    df.loc[:, "neutral_sub"] = neutralize(df, [PREDICTION_NAME],
                                          feature_cols)[PREDICTION_NAME]
    scores = df.groupby("era").apply(
        lambda x: np.corrcoef(x["neutral_sub"].rank(pct=True, method="first"), x[TARGET_NAME])).mean()
    return np.mean(scores)

def compute_val_mmc(valid_df : pd.DataFrame):    
    # MMC over validation
    mmc_scores = []
    corr_scores = []
    for _, x in valid_df.groupby("era"):
        series = neutralize_series(pd.Series(unif(x[PREDICTION_NAME])),
                                   pd.Series(unif(x[EXAMPLE_PRED])))
        mmc_scores.append(np.cov(series, x[TARGET_NAME])[0, 1] / (0.29 ** 2))
        corr_scores.append(np.corrcoef(unif(x[PREDICTION_NAME]).rank(pct=True, method="first"), x[TARGET_NAME]))

    val_mmc_mean = np.mean(mmc_scores)
    val_mmc_std = np.std(mmc_scores)
    val_mmc_sharpe = val_mmc_mean / val_mmc_std
    corr_plus_mmcs = [c + m for c, m in zip(corr_scores, mmc_scores)]
    corr_plus_mmc_sharpe = np.mean(corr_plus_mmcs) / np.std(corr_plus_mmcs)
    corr_plus_mmc_mean = np.mean(corr_plus_mmcs)

    logger.info("MMC Mean = {:.6f}, MMC Std = {:.6f}, CORR+MMC Sharpe = {:.4f}".format(val_mmc_mean, val_mmc_std, corr_plus_mmc_sharpe))

    # Check correlation with example predictions
    corr_with_example_preds = np.corrcoef(valid_df[EXAMPLE_PRED].rank(pct=True, method="first"),
                                          valid_df[PREDICTION_NAME].rank(pct=True, method="first"))[0, 1]
    logger.info("Corr with example preds: {:.4f}".format(corr_with_example_preds))
    
    return val_mmc_mean, val_mmc_std, corr_plus_mmc_sharpe, corr_with_example_preds
    
def score_summary(valid_df : pd.DataFrame):
    score_df = {}
    
    try:
        score_df['correlation'] = compute_val_corr(valid_df)
    except:
        print('ERR: computing correlation')
    try:
        score_df['corr_sharpe'], score_df['corr_mean'], score_df['corr_std'], score_df['max_drawdown'] = compute_val_sharpe(valid_df)
    except:
        print('ERR: computing sharpe')
    try:
        score_df['feature_exposure'], score_df['max_feature_exposure'] = compute_val_feature_exposure(valid_df)
    except:
        print('ERR: computing feature exposure')
    try:
        score_df['mmc_mean'], score_df['mmc_std'], score_df['corr_mmc_sharpe'], score_df['corr_with_example_xgb'] = compute_val_mmc(valid_df)
    except:
        print('ERR: computing MMC')
    
    return pd.DataFrame.from_dict(score_df, orient='index')

In [ ]:
valid_df = valid4score(validation_data, pred, load_example=False, save=False)

# scores
score_df = pd.DataFrame()
print('------------------')
print('ALL:')
print('------------------')
all_ = score_summary(valid_df).rename(columns={0: 'all'})

print('------------------')
print('VALID 1:')
print('------------------')
val1_ = score_summary(valid_df.query('era < 150')).rename(columns={0: 'val1'})

print('------------------')
print('VALID 2:')
print('------------------')
val2_ = score_summary(valid_df.query('era > 150')).rename(columns={0: 'val2'})

In [ ]:
# scores
score_df = pd.concat([all_, val1_, val2_], axis=1)
score_df.style.background_gradient(cmap='viridis', axis=0)
